<a href="https://colab.research.google.com/github/kadirov1194/ShoxruxProject/blob/main/qorakoz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 23.7 MB/s eta 0:00:00


In [2]:
import os
import shutil
import random
!pip install tqdm --upgrade
from tqdm.notebook import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [5]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images
    

    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    
    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue
      shutil.copy2(path + filex + '.jpg',f"{train_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{train_path_label}/" + filex + '.txt')
        
    

    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")
    
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')
            
        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")
    
        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")
    


    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue
      # print("running")
      shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg' )
      shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + '.txt')

    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")
    
    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

### for label_tag 
train_test_split('/content/drive/MyDrive/qorakoz/data/') ### without negative images 
#train_test_split('./data/','./negative_images/') ### if you want to feed negative images

------ PROCESS STARTED -------
--- This folder has a total number of 5205 images---


  0%|          | 0/4164 [00:00<?, ?it/s]

------ Training data created with 80% split 4164 images -------


  0%|          | 0/1041 [00:00<?, ?it/s]

------ Testing data created with a total of 1041 images ----------
------ TASK COMPLETED -------


In [6]:
# I will write the contents of the cell to a file
%%writefile /content/drive/MyDrive/qorakoz/dataset.yaml 

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
#path:  # dataset root dir
train: /content/yolo_data/images/train # train images (relative to 'path')
val: /content/yolo_data/images/val # val images (relative to 'path')
test:  # test images (optional)

# number of classes
nc: 3

names: [ '1','2','3']  

Overwriting /content/drive/MyDrive/qorakoz/dataset.yaml


In [7]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.52 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 25.8/78.2 GB disk)


In [8]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/qorakoz/dataset.yaml epochs=50 imgsz=640 batch=8 project=/content/drive/MyDrive/qorakoz/training_results name=qora

100% 21.5M/21.5M [00:00<00:00, 44.4MB/s]
Ultralytics YOLOv8.0.52 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/qorakoz/dataset.yaml, epochs=50, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/qorakoz/training_results, name=qora, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=Fa

In [9]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/qorakoz/training_results/qora/weights/best.pt conf=0.55 source=/content/drive/MyDrive/qorakoz/test_images save=True

Ultralytics YOLOv8.0.52 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

image 1/295 /content/drive/MyDrive/qorakoz/test_images/432_jpg.rf.819f2d34b696ce63851f6eacbab1e348.jpg: 480x640 1 1, 1 2, 2 3s, 24.8ms
image 2/295 /content/drive/MyDrive/qorakoz/test_images/433_jpg.rf.cfe3a641e016ac7c2158619a27fd815c.jpg: 480x640 1 1, 1 2, 1 3, 14.1ms
image 3/295 /content/drive/MyDrive/qorakoz/test_images/435_jpg.rf.11771456a1a556edd431202b7dc37a1a.jpg: 480x640 1 1, 1 2, 49.6ms
image 4/295 /content/drive/MyDrive/qorakoz/test_images/437_jpg.rf.eb38ebccdde248b1391240a61c9ef225.jpg: 480x640 1 1, 1 2, 14.2ms
image 5/295 /content/drive/MyDrive/qorakoz/test_images/438_jpg.rf.536d653afeea9ca7bf047a4f391bb0dc.jpg: 480x640 1 1, 1 2, 1 3, 13.9ms
image 6/295 /content/drive/MyDrive/qorakoz/test_images/442_jpg.rf.99abcc3911ce3dc4f8942793dac50198.jpg: 480x640 1 1, 1 2, 3 3s, 13.9ms
image 7/295 /content/drive/MyDrive/

In [10]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/qorakoz/output

In [11]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/qorakoz/training_results/qora/weights/best.pt conf=0.55 source=/content/drive/MyDrive/qorakoz/vid save=True

Ultralytics YOLOv8.0.52 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126745 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 1 2, 19.1ms
video 1/1 (2/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 1 2, 12.0ms
video 1/1 (3/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 1 2, 11.8ms
video 1/1 (4/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 1 2, 12.0ms
video 1/1 (5/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 2 2s, 11.9ms
video 1/1 (6/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 2 2s, 11.9ms
video 1/1 (7/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 2 2s, 11.9ms
video 1/1 (8/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 2 2s, 12.0ms
video 1/1 (9/273) /content/drive/MyDrive/qorakoz/vid/IMG_6416.MOV: 384x640 (no detections), 11.7ms
video 1/1 (10/273) /content/driv

In [13]:
!cp -r /content/runs/detect/predict2 /content/drive/MyDrive/qorakoz/output